In [1]:
import os
os.sys.path.append('../..')
import h5py
from IPython.display import display

import project.download_content as content

modelpath = os.path.join(os.path.abspath(content.DATAPATH), 'MODEL') 


val_files = [os.path.join(modelpath, f'39_classes_300x300_cross_val_{i}.h5')
             for i in range(8)]

test_files = [os.path.join(modelpath, f'39_classes_300x300_test_{i}.h5')
              for i in range(8)]

In [2]:
%%time
for files in [test_files, val_files]:
    for f_path in files:
        with h5py.File(f_path, 'a') as f:
            l = list(f.keys())
            if not 'batches' in l:
                unique_ids = set(i[:36] for i in l)

                batches = []

                for id in unique_ids:
                    batches.append([f"{id}-x".encode("ascii", "ignore"),
                                    f"{id}-y".encode("ascii", "ignore"),
                                    f"{id}-info".encode("ascii", "ignore"),
                                    f"{id}-bboxes".encode("ascii", "ignore")])

                f.create_dataset(name='batches',
                                 shape=(len(batches), 4),
                                 data=batches,
                                 dtype=h5py.special_dtype(vlen=str),
                                 compression='gzip',
                                 compression_opts=9)

CPU times: user 743 µs, sys: 4.26 ms, total: 5 ms
Wall time: 4.96 ms


In [3]:
%%time
total_batches = 0
total_images = 0

for f_path in val_files + test_files:
    with h5py.File(f_path, 'r') as f:
        l = list(f.keys())
        first = f['batches'][0]
        last = f['batches'][-1]
        partial_images = 0
        
        for x_ref, _, _, _ in f['batches']:
            temp_count = f[x_ref].shape[0]
            partial_images += temp_count
            total_images += temp_count
            
        total_batches += len(f["batches"])
        
        f_path = '...' + f_path[-40:]
        
        print('file', f_path, 'has', len(l), 'datasets')
        print('The image dataset has', len(f['batches']), 'batches',
              f'({len(f["batches"])} x 4) + 1 =', (len(f["batches"])*4 + 1))
        print('This dataset has a total of', partial_images, 'images')
        
        print('  the first image x ref in', f_path, "is present?", 
              (first[0] in l))
        print('  the first image y ref in', f_path, "is present?", 
              (first[1] in l))
        print('   the last image x ref in', f_path, "is present?", 
              (last[0] in l))
        print('   the last image y ref in', f_path, "is present?", 
              (last[1] in l), end='\n\n')
        
        
print(f'A total of {total_batches} batches are going to be used in train')
print(f'there are {total_images} images among this batches')

file .../MODEL/39_classes_300x300_cross_val_0.h5 has 65 datasets
The image dataset has 16 batches (16 x 4) + 1 = 65
This dataset has a total of 4800 images
  the first image x ref in .../MODEL/39_classes_300x300_cross_val_0.h5 is present? True
  the first image y ref in .../MODEL/39_classes_300x300_cross_val_0.h5 is present? True
   the last image x ref in .../MODEL/39_classes_300x300_cross_val_0.h5 is present? True
   the last image y ref in .../MODEL/39_classes_300x300_cross_val_0.h5 is present? True

file .../MODEL/39_classes_300x300_cross_val_1.h5 has 65 datasets
The image dataset has 16 batches (16 x 4) + 1 = 65
This dataset has a total of 4800 images
  the first image x ref in .../MODEL/39_classes_300x300_cross_val_1.h5 is present? True
  the first image y ref in .../MODEL/39_classes_300x300_cross_val_1.h5 is present? True
   the last image x ref in .../MODEL/39_classes_300x300_cross_val_1.h5 is present? True
   the last image y ref in .../MODEL/39_classes_300x300_cross_val_1.h5 